In [29]:
import pandas as pd
import requests
import numpy as np
from sklearn.ensemble import IsolationForest

# Load wallet list
wallet_df = pd.read_excel("/content/Wallet id.xlsx")
wallets = wallet_df['wallet_id'].tolist()

# Helper function to simulate fetching Compound data
def fetch_compound_data(wallet_id):
    # Replace with real query logic via The Graph or Debank
    np.random.seed(int(wallet_id[:6], 16) % 100000)  # Seed for consistency
    return {
        'num_borrows': np.random.randint(0, 10),
        'total_borrow_amount': np.random.uniform(100, 5000),
        'num_repayments': np.random.randint(0, 10),
        'total_repayment_amount': np.random.uniform(100, 5000),
        'liquidation_events': np.random.randint(0, 3),
        'max_collateral': np.random.uniform(500, 10000),
        'avg_collateralization_ratio': np.random.uniform(1.0, 3.5),
        'activity_days': np.random.randint(10, 400),
    }

# Step 1: Gather features
data = []
for wallet in wallets:
    features = fetch_compound_data(wallet)
    features['wallet_id'] = wallet
    data.append(features)

features_df = pd.DataFrame(data)

# Step 2: Feature Engineering
features_df['repayment_ratio'] = features_df['total_repayment_amount'] / features_df['total_borrow_amount'].replace(0, 1)
features_df['borrow_activity_score'] = features_df['num_borrows'] / features_df['activity_days']
features_df['liquidation_penalty'] = features_df['liquidation_events'] * 100

# Step 3: Normalization
def min_max_scale(series):
    return (series - series.min()) / (series.max() - series.min())

features_df['behavior_score'] = (
    min_max_scale(features_df['repayment_ratio']) * 0.5 +
    min_max_scale(features_df['borrow_activity_score']) * 0.3 -
    min_max_scale(features_df['liquidation_penalty']) * 0.2
)

features_df['collateral_score'] = (
    min_max_scale(features_df['max_collateral']) * 0.4 +
    min_max_scale(features_df['avg_collateralization_ratio']) * 0.6
)

features_df['final_score'] = (
    features_df['behavior_score'] * 0.5 +
    features_df['collateral_score'] * 0.5
)

# Scale to 0–1000
features_df['score'] = (features_df['final_score'] * 1000).clip(0, 1000).astype(int)

# Step 4: Outlier detection (optional)
clf = IsolationForest(contamination=0.05)
features_df['outlier'] = clf.fit_predict(features_df[['behavior_score', 'collateral_score']])
features_df['score'] = np.where(features_df['outlier'] == -1, features_df['score'] * 0.7, features_df['score']).astype(int)

# Step 5: Save output
output_df = features_df[['wallet_id', 'score']]
output_df.to_csv("wallet_risk_scores.csv", index=False)
print("✅ Risk scores saved to wallet_risk_scores.csv")


✅ Risk scores saved to wallet_risk_scores.csv


In [30]:
wallets_df.shape

(103, 1)